## Question 1: TwitterReplyPercentage

In [6]:
import tweepy
from twitter_config import config

In [7]:
def get_reply_percentage(config, username, max_tweets=100):

    auth = tweepy.OAuthHandler(config['consumer_key'], config['consumer_secret'])
    auth.set_access_token(config['access_token'], config['access_token_secret'])
    api = tweepy.API(auth)
    
    tweets = 0 #keep a counter in case the use doesn't have as mas as the max_tweets we give
    reply_status = 0
    
    for status in tweepy.Cursor(api.user_timeline, id=username).items(max_tweets):
        tweets += 1
        if status.in_reply_to_status_id is not None:
            print (status.text)
            reply_status += 1
    return round(((reply_status/tweets)*100),1)

In [8]:
screen_name = 'jimmyfallon'
max_tweets = 500

percentage = get_reply_percentage(config, screen_name, max_tweets)
print ("The percentage of the tweets that were replies of the user {} is {} % ".format(screen_name, percentage))

@CaillouPettis Sure.
@Stranger_Things @JoseAltuve27 @KeeganMKey Plus we've got music from the great @Elbow!
I’m sorry to my college roommate. I’m the one who ate all your Bagel Bites. It was a pizza emergency. #WorldSorries
You Are What You Eat. Which Means You Eat Awesome. #MondayMotivation #FallonTonight
Swipe Right On Yourself. #MondayMotivation #FallonTonight
@HillaryClinton @MileyCyrus Hilary was great. So was Miley!! Wow!!
Thanks again to @kj_apa, @colesprouse, @CamilaMendes, @lilireinhart, and @madelainepetsch! Can't wait for Season 2!!
@ddlovato You did?!?! You did?!???? You did!!!! https://t.co/dVjejQBbRg
@catheb You will love it!! Great monologue, great guests and the Spike thing will be memorable.
Plus a Spike Jonze live dance performance featuring Mia Wasikowska &amp; @lakeithlakeith! Don't miss! #FallonTonight
@CecilyMcLeod Thank you.
.@U2 performs their brand new single "You’re The Best Thing About Me" https://t.co/5TZSuy74I5 #U2onFallon #FallonTonight
@U2 Stay up!!! DVR!

## Question 2: GutenbergProject

In [9]:
import requests
from bs4 import BeautifulSoup

To find ALL the books listed in gutenberg.org we need to find all the urls of the page results. I achieve that by finding the statusline and specifically the one with title "Go to the next page of results.".

Afterwards the titles of the books are collected by finding the booklink class elements that contain class subtitle (to find the author) and class title element that contains the actual title of the book. 

In [10]:
def find_titles(url):
    titles=[]
    while True:
        try:
            r = requests.get("https://www.gutenberg.org" + url)

        except requests.exceptions.RequestException as rex:
            print("Unable to get {article_url} reason {rex}")
            break
        else:
            html = r.content
            soup = BeautifulSoup(html, 'html.parser')
            review_results = soup.find_all("li", {"class": {"booklink"}})
            for result in review_results:
                if result.find("span", class_="subtitle") is not None:
                    author = result.find('span', class_="subtitle").get_text()
                    if "Charles Dickens" in author:
                        for x in result.find_all(class_="title"):
                            titles.append(x.get_text())

            statuslinks = soup.find_all("li", {"class": {"statusline"}})
            for link in statuslinks:
                next_page = link.find_all("a", {"title": "Go to the next page of results."}, href=True)
            if not next_page:
                break
            else:
                for i in next_page:
                    url = i["href"]
    return titles

In [11]:
url_link = "/ebooks/search/?&query=charles+dickens"
books = set(find_titles(url_link))
for i, title in enumerate(books):
    print(i, title)

0 David Copperfield
1 George Silverman's Explanation
2 A Christmas Carol in Prose; Being a Ghost Story of Christmas
3 The Letters of Charles Dickens
4 Tales from Dickens
5 Holiday Romance
6 The Holly-Tree
7 Some Christmas Stories
8 Schetsen uit Amerika en Tafereelen uit Italië (Dutch)
9 The Battle of Life. A Love Story
10 All the Year Round
11 Pearl-Fishing; Choice Stories from Dickens' Household Words; First Series
12 A Message from the Sea. Finnish (Finnish)
13 Sketches of Young Gentlemen
14 David Copperfield - Tome II (French)
15 David Copperfield - Tome I (French)
16 Pearl-Fishing; Choice Stories from Dickens' Household Words; Second Series
17 A Christmas carol. Dutch (Dutch)
18 The cricket on the hearth. Finnish (Finnish)
19 No thoroughfare. Finnish (Finnish)
20 A House to Let
21 Captain Boldheart & the Latin-Grammar Master
22 Cruikshank's Water Colours
23 Le magasin d'antiquités, Tome I (French)
24 The Perils of Certain English Prisoners
25 Doctor Marigold
26 Pictures from Italy


## Question 3: AChristmasCarol

In [12]:
import re
from bs4 import BeautifulSoup
import requests

I assume that a sentence is a period and may include secondary sentences such as those included in a dialogue inside quotes or those that follow a colon. 
The regular expression used is explained as follows:
A whitespace character (\s) that is preceded by a . or ? or ! characters (positive lookbehind (?<=\.|\?|\!)) and is not preceded by any single character from A-Z and a-z and a . character(negative lookbehind (?<![A-Z][a-z]\.) and is not preceded by a word character, a . character and another word character (negative lookbehind (?<!\w\.\w.)). 

In [13]:
r = requests.get("https://www.gutenberg.org/cache/epub/30368/pg30368.txt")
text = r.content
soup = BeautifulSoup(text, "lxml")
carol = soup.find('body').get_text()

sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', carol)
count = 1
for sent in sentences:
    if 'THE END.' in sent:
        break
    if 'Scrooge' in sent:
        # To remove illustration comments in [] that are included in the text
        pretty_sentence = re.sub('(\[[^]]*])', ' ', sent)
        # To remove more than one whitespace caused due to format of the text
        pretty_sentence = re.sub('\s+', ' ', pretty_sentence)
        print(count, pretty_sentence)
        count += 1

1 Scrooge signed it: and Scrooge's name was good upon 'Change, for anything he chose to put his hand to.
2  Scrooge knew he was dead?
3  Scrooge and he were partners for I don't know how many years.
4 Scrooge was his sole executor, his sole administrator, his sole assign, his sole residuary legatee, his sole friend and sole mourner.
5 And even Scrooge was not so dreadfully cut up by the sad event, but that he was an excellent man of business on the very day of the funeral, and solemnised it with an undoubted bargain.
6  Scrooge never painted out Old Marley's name.
7 There it stood, years afterwards, above the warehouse door: Scrooge and Marley.
8  The firm was known as Scrooge and Marley.
9 Sometimes people new to the business called Scrooge Scrooge, and sometimes Marley, but he answered to both names: it was all the same to him.
10 But he was a tight-fisted hand at the grindstone, Scrooge!
11  External heat and cold had little influence on Scrooge.
12 They often "came down" handsomely